In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [2]:
from sqlalchemy import create_engine
!pip install PyMySql
import pymysql
import sqlalchemy
import json
import pandas as pd
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [3]:
tip = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json') as fl:
    for i, line in enumerate(fl):
        tip.append(json.loads(line))
dftip = pd.DataFrame(tip)
dftip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [5]:
user = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json') as fl:
    for i, line in enumerate(fl):
        user.append(json.loads(line))
dfuser = pd.DataFrame(user)
dfuser.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [3]:
business = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))
dfbusiness = pd.DataFrame(business)
dfbusiness.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
checkin = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json') as fl:
    for i, line in enumerate(fl):
        checkin.append(json.loads(line))
dfcheckin = pd.DataFrame(checkin)
dfcheckin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [4]:
review = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        review.append(json.loads(line))
dfreview = pd.DataFrame(review)
dfreview['date'] = pd.to_datetime(dfreview['date'], format='%Y-%m-%d %H:%M:%S')

# Extract the day of the week and create a new column
dfreview['Day_of_Week'] = dfreview['date'].dt.strftime('%A')
dfreview['Year'] = dfreview['date'].dt.year
dfreview.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Day_of_Week,Year
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Saturday,2018
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,Tuesday,2012
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,Wednesday,2014
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Sunday,2015
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Saturday,2017


In [5]:
def expand_multivalue_column(df, col_name):
    # Convert string representation of dictionaries into actual dictionaries (if needed)
    expanded_df = df[col_name].apply(pd.Series)
    
    # For columns within the expanded dataframe that are still string representations of dictionaries
    for sub_col in expanded_df.columns:
        if isinstance(expanded_df[sub_col].iloc[0], str) and expanded_df[sub_col].iloc[0].startswith("{"):
            sub_df = expanded_df[sub_col].map(eval).apply(pd.Series)
            sub_df = sub_df.add_prefix(f'{sub_col}_')
            expanded_df = pd.concat([expanded_df.drop([sub_col], axis=1), sub_df], axis=1)
    
    # Add the business_id column for the foreign key reference
    expanded_df['business_id'] = df['business_id']
    
    return expanded_df

# Expand the attributes and hours columns into separate tables
attributes_table = expand_multivalue_column(dfbusiness, 'attributes')
hours_table = expand_multivalue_column(dfbusiness, 'hours')

# Drop the 'attributes' and 'hours' columns from the original business_df
dfbusiness = dfbusiness.drop(columns=['attributes', 'hours'])

print("Original Business Table:")
print(dfbusiness.head())
print("\nAttributes Table:")
print(attributes_table.head())
print("\nHours Table:")
print(hours_table.head())

Original Business Table:
              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            

In [6]:
def convert_price_range(price_range):
    if price_range == '1':
        return 1
    elif price_range == '2':
        return 2
    elif price_range == '3':
        return 3
    elif price_range == '4':
        return 4
    elif price_range == '0':
        return 1
    else:
        return 1

def convert_boolean(boolean):
    if boolean == 'True':
        return 1
    else:
        return 0
    
    
attributes_table['RestaurantsPriceRange'] = attributes_table['RestaurantsPriceRange2'].apply(convert_price_range) 
attributes_table['Dogs_Allowed'] = attributes_table['DogsAllowed'].apply(convert_boolean)

In [7]:
# Split the 'categories' column by commas and transform to long format
categories_table = dfbusiness['categories'].str.split(', ', expand=True).stack().reset_index()
categories_table.columns = ['business_id', 'category_number', 'category']
categories_table['business_id'] = dfbusiness['business_id']
categories_table = categories_table[['business_id', 'category']]  # Drop the category_number column

# Drop the 'categories' column from the original business_df
dfbusiness = dfbusiness.drop(columns=['categories'])

print("Original Business Table:")
print(dfbusiness)
print("\nCategories Table:")
print(categories_table)

Original Business Table:
                   business_id                      name  \
0       Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1       mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2       tUFrWirKiKi_TAnsVWINQQ                    Target   
3       MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4       mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
...                        ...                       ...   
150341  IUQopTMmYQG-qRtBk-8QnA              Binh's Nails   
150342  c8GjPIOTGVmIemT7j5_SyQ      Wild Birds Unlimited   
150343  _QAMST-NrQobXduilWEqSw         Claire's Boutique   
150344  mtGm22y5c2UHNXDFAjaPNw  Cyclery & Fitness Center   
150345  jV_XOycEzSlTx-65W906pg                   Sic Ink   

                                address           city state postal_code  \
0                1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1       87 Grasso Plaza Shopping Center         Affton    MO       63123   
2                  5255 E 

In [13]:
import ast
Parking_df = pd.DataFrame(attributes_table['BusinessParking'])
def safe_eval(text):
    try:
        return ast.literal_eval(text)
    except (SyntaxError, ValueError):
        return None
 
# Apply the custom function to handle the text data
Parking_df['BusinessParking'] = Parking_df['BusinessParking'].apply(safe_eval)
Parking_df = pd.concat([Parking_df, Parking_df['BusinessParking'].apply(pd.Series)], axis=1)
Parking_df = Parking_df.drop(columns='BusinessParking')
Parking_df['business_id'] = attributes_table['business_id']
print(Parking_df.head())

  garage street validated    lot  valet             business_id
0    NaN    NaN       NaN    NaN    NaN  Pns2l4eNsfO8kk83dixA6A
1    NaN    NaN       NaN    NaN    NaN  mpf3x-BjTdTEA3yCZrAYPw
2  False  False     False   True  False  tUFrWirKiKi_TAnsVWINQQ
3  False   True     False  False  False  MTSW4McQd7CbVtyjqoe9mw
4   None   None      None   True  False  mWMc6_wTdE0EUBKIGXDVfA


In [14]:
Ambience_df = pd.DataFrame(attributes_table['Ambience'])
 
# Apply the custom function to handle the text data
Ambience_df['Ambience'] = Ambience_df['Ambience'].apply(safe_eval)
Ambience_df = pd.concat([Ambience_df, Ambience_df['Ambience'].apply(pd.Series)], axis=1)
Ambience_df = Ambience_df.drop(columns='Ambience')
Ambience_df['business_id'] = attributes_table['business_id']
print(Ambience_df.head())

  romantic intimate touristy hipster divey classy trendy upscale casual  \
0      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
1      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
2      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
3      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
4      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   

              business_id  
0  Pns2l4eNsfO8kk83dixA6A  
1  mpf3x-BjTdTEA3yCZrAYPw  
2  tUFrWirKiKi_TAnsVWINQQ  
3  MTSW4McQd7CbVtyjqoe9mw  
4  mWMc6_wTdE0EUBKIGXDVfA  


In [15]:
GoodForMeal_df = pd.DataFrame(attributes_table['GoodForMeal'])
 
# Apply the custom function to handle the text data
GoodForMeal_df['GoodForMeal'] = GoodForMeal_df['GoodForMeal'].apply(safe_eval)
GoodForMeal_df = pd.concat([GoodForMeal_df, GoodForMeal_df['GoodForMeal'].apply(pd.Series)], axis=1)
GoodForMeal_df = GoodForMeal_df.drop(columns='GoodForMeal')
GoodForMeal_df['business_id'] = attributes_table['business_id']
print(GoodForMeal_df.head())

  dessert latenight lunch dinner brunch breakfast             business_id
0     NaN       NaN   NaN    NaN    NaN       NaN  Pns2l4eNsfO8kk83dixA6A
1     NaN       NaN   NaN    NaN    NaN       NaN  mpf3x-BjTdTEA3yCZrAYPw
2     NaN       NaN   NaN    NaN    NaN       NaN  tUFrWirKiKi_TAnsVWINQQ
3     NaN       NaN   NaN    NaN    NaN       NaN  MTSW4McQd7CbVtyjqoe9mw
4     NaN       NaN   NaN    NaN    NaN       NaN  mWMc6_wTdE0EUBKIGXDVfA


In [ ]:
Music_df = pd.DataFrame(attributes_table['Music'])
 
# Apply the custom function to handle the text data
Music_df['Music'] = Music_df['Music'].apply(safe_eval)
Music_df = pd.concat([Music_df, Music_df['Music'].apply(pd.Series)], axis=1)
Music_df = Music_df.drop(columns='Music')
Music_df['business_id'] = attributes_table['business_id']
print(Music_df.head())

In [ ]:
BestNights_df = pd.DataFrame(attributes_table['BestNights'])
 
# Apply the custom function to handle the text data
BestNights_df['BestNights'] = BestNights_df['BestNights'].apply(safe_eval)
BestNights_df = pd.concat([BestNights_df, BestNights_df['BestNights'].apply(pd.Series)], axis=1)
BestNights_df = BestNights_df.drop(columns='BestNights')
BestNights_df['business_id'] = attributes_table['business_id']
print(BestNights_df.head())

In [ ]:
HairSpecializesIn_df = pd.DataFrame(attributes_table['HairSpecializesIn'])
 
# Apply the custom function to handle the text data
HairSpecializesIn_df['HairSpecializesIn'] = HairSpecializesIn_df['HairSpecializesIn'].apply(safe_eval)
HairSpecializesIn_df = pd.concat([HairSpecializesIn_df, HairSpecializesIn_df['HairSpecializesIn'].apply(pd.Series)], axis=1)
HairSpecializesIn_df = HairSpecializesIn_df.drop(columns='HairSpecializesIn')
HairSpecializesIn_df['business_id'] = attributes_table['business_id']
print(HairSpecializesIn_df.head())

In [ ]:
DietaryRestrictions_df = pd.DataFrame(attributes_table['DietaryRestrictions'])
 
# Apply the custom function to handle the text data
DietaryRestrictions_df['DietaryRestrictions'] = DietaryRestrictions_df['DietaryRestrictions'].apply(safe_eval)
DietaryRestrictions_df = pd.concat([DietaryRestrictions_df, DietaryRestrictions_df['DietaryRestrictions'].apply(pd.Series)], axis=1)
DietaryRestrictions_df = DietaryRestrictions_df.drop(columns='DietaryRestrictions')
DietaryRestrictions_df['business_id'] = attributes_table['business_id']
print(DietaryRestrictions_df.head())

In [22]:
#The code initially written for the city of "Chicago" did not render results due 
#to lack of data in the yelp database. I have retained the same queries in python form 
#but inorder to see the visual data I have updated this to "Santa Barbara".

In [8]:
# Businesses in Santa Barbara
condition1 = dfbusiness['city'] == 'Santa Barbara'
SantaBarbara_businesses = dfbusiness[condition1]
# Categories restricted to Restaurants
condition2 = categories_table['category'] == 'Restaurants'
restaurants_businesses = categories_table[condition2]
#Joined DF of Resaturants in Santa Barbara
santabarbara_restaurants_df = pd.merge(SantaBarbara_businesses, restaurants_businesses, on='business_id')

In [9]:
filter_values = ['Bars','Bakeries','Coffee & Tea','Coffee Roasteries','Desserts','Empanadas','Food','Food Stands','Food Tours','Food Trucks','Ice Cream & Frozen Yogurt','Indian','Kebab','Restaurants']
categories_businesses = categories_table[categories_table['category'].isin(filter_values)]
business_categories = pd.merge(dfbusiness,categories_businesses, on='business_id')
print(business_categories.head())

              business_id                            name  \
0  M0XSSHqrASOnhgbWDJIpQA                  Herb Import Co   
1  8wGISYjYkE2tSqn3cDMu8A                Nifty Car Rental   
2  WKMJwqnfZKsAae75RMP6jA  Roast Coffeehouse and Wine Bar   
3  qhDdDeI3K4jy2KyzwFN53w      Barnes & Noble Booksellers   
4  4iRzR7OaS-QaSXuvYxEGKA                       Super Dog   

                    address          city state postal_code   latitude  \
0              712 Adams St   New Orleans    LA       70118  29.941468   
1           1241 Airline Dr        Kenner    LA       70062  29.981183   
2       10359 104 Street NW      Edmonton    AB     T5J 1B9  53.546045   
3  8675 River Crossing Blvd  Indianapolis    IN       46240  39.913820   
4      1160 Gallatin Pike S     Nashville    TN       37115  36.248570   

    longitude  stars  review_count  is_open      category  
0  -90.129953    4.0             5        1   Restaurants  
1  -90.254012    3.5            14        1          Food  
2 -113.4

In [11]:
# 1. Understanding day-based reviews count to predict demand days for restaurants in santa barbara
santabarbara_restaurants_reviews_df = pd.merge(santabarbara_restaurants_df, dfreview, on='business_id')
grouped_df = santabarbara_restaurants_reviews_df.groupby('Day_of_Week').size().reset_index(name='Count')
sorted_grouped_df = grouped_df.sort_values(by='Count', ascending=False)
print(sorted_grouped_df)


  Day_of_Week  Count
1      Monday   2961
3      Sunday   2900
5     Tuesday   2610
2    Saturday   2525
6   Wednesday   2483
4    Thursday   2394
0      Friday   2364


In [12]:
# 2. Top businesses by weighted average:
grouped_df5 = santabarbara_restaurants_df.groupby('name').agg(
    weighted_avg_rating=('stars', lambda x: (x * santabarbara_restaurants_df.loc[x.index, 'review_count']).sum() / santabarbara_restaurants_df.loc[x.index, 'review_count'].sum())
)
print(grouped_df5.head())

                         weighted_avg_rating
name                                        
19 Blue Salon                            4.5
4 Eggs & Pizza                           5.0
805 Deli                                 4.0
Ace Plumbing                             5.0
Acupuncture Integration                  5.0


In [13]:
# 3. Restaurants ratings over the years for restaurants in santa barbara

grouped_df2 = santabarbara_restaurants_reviews_df.groupby(['name','Year'])['stars_y'].mean().reset_index(name='Average_Rating').sort_values(by=['name', 'Year'])
print(grouped_df2.head(20))

              name  Year  Average_Rating
0    19 Blue Salon  2008        4.666667
1    19 Blue Salon  2009        4.666667
2    19 Blue Salon  2010        4.538462
3    19 Blue Salon  2011        4.000000
4    19 Blue Salon  2012        4.600000
5    19 Blue Salon  2013        4.500000
6    19 Blue Salon  2014        4.833333
7    19 Blue Salon  2015        5.000000
8    19 Blue Salon  2016        4.416667
9    19 Blue Salon  2017        4.800000
10   19 Blue Salon  2018        5.000000
11   19 Blue Salon  2019        4.500000
12   19 Blue Salon  2020        3.000000
13  4 Eggs & Pizza  2021        4.813559
14  4 Eggs & Pizza  2022        5.000000
15        805 Deli  2010        5.000000
16        805 Deli  2011        5.000000
17        805 Deli  2012        3.800000
18        805 Deli  2013        4.285714
19        805 Deli  2014        3.000000


In [14]:
# 4. Average number of reviews per city:
business_categories_reviews = pd.merge(business_categories,dfreview, on='business_id')
grouped_df3 = business_categories_reviews.groupby('city')['review_id'].count().sort_values(ascending=False)
print(grouped_df3.head())

city
Philadelphia    157002
New Orleans     114020
Tampa            74961
Nashville        74738
Tucson           66655
Name: review_id, dtype: int64


In [15]:
# 5. Calculate the average ratings of restaurant categories:

#Joined DF of Resaturant Categories in Santa Barbara
santabarbara_categories_df = pd.merge(SantaBarbara_businesses, categories_businesses, on='business_id')
santabarbara_categories_grouped_df = santabarbara_categories_df.groupby('category')['stars'].mean().reset_index(name='average_rating')
print(santabarbara_categories_grouped_df)

                     category  average_rating
0                    Bakeries        4.041667
1                        Bars        4.113333
2                Coffee & Tea        4.052632
3           Coffee Roasteries        4.500000
4                    Desserts        4.192308
5                   Empanadas        4.000000
6                        Food        4.136054
7                 Food Stands        3.833333
8                 Food Trucks        4.041667
9   Ice Cream & Frozen Yogurt        4.115385
10                     Indian        3.000000
11                      Kebab        1.500000
12                Restaurants        4.054422


In [16]:
# 6. Calculate the average pricing of restaurants:
santabarbara_categories_attributes_df = pd.merge(santabarbara_categories_df, attributes_table, on='business_id')
santabarbara_categories_attributes_group_df = santabarbara_categories_attributes_df.groupby('category').agg(
    Max_Price=('RestaurantsPriceRange', 'max'),
    Min_Price=('RestaurantsPriceRange', 'min'),
    Avg_Price=('RestaurantsPriceRange', 'mean')
).reset_index()
print(santabarbara_categories_attributes_group_df)

                     category  Max_Price  Min_Price  Avg_Price
0                    Bakeries          3          1   1.541667
1                        Bars          3          1   1.453333
2                Coffee & Tea          3          1   1.368421
3           Coffee Roasteries          1          1   1.000000
4                    Desserts          4          1   1.692308
5                   Empanadas          1          1   1.000000
6                        Food          3          1   1.401361
7                 Food Stands          1          1   1.000000
8                 Food Trucks          3          1   1.666667
9   Ice Cream & Frozen Yogurt          3          1   1.461538
10                     Indian          2          1   1.333333
11                      Kebab          2          2   2.000000
12                Restaurants          4          1   1.482993


In [17]:
# 7. Analysis on pet friendliness :
grouped_df4 = santabarbara_categories_attributes_df.groupby('category').agg(
    pet_friendly=('Dogs_Allowed', 'sum'),
    percentage_pet_friendly=('Dogs_Allowed', lambda x: (x.mean() * 100)))
print(grouped_df4)

                           pet_friendly  percentage_pet_friendly
category                                                        
Bakeries                              2                 8.333333
Bars                                  5                 6.666667
Coffee & Tea                          5                13.157895
Coffee Roasteries                     0                 0.000000
Desserts                              2                15.384615
Empanadas                             0                 0.000000
Food                                  9                 6.122449
Food Stands                           0                 0.000000
Food Trucks                           0                 0.000000
Ice Cream & Frozen Yogurt             2                15.384615
Indian                                0                 0.000000
Kebab                                 0                 0.000000
Restaurants                          25                 8.503401


In [21]:
# 8. Distribution of ratings, looking at the number of restaurants that fall into different rating ranges
def categorize_stars(stars):
    if stars >= 4.5:
        return 'Highly Rated (4.5 - 5)'
    elif stars >= 3.5:
        return 'Moderately Rated (3.5 - 4.49)'
    elif stars >= 2.5:
        return 'Average Rated (2.5 - 3.49)'
    else:
        return 'Low Rated (1 - 2.49)'
    
    
business_categories_reviews['rating_range'] = business_categories_reviews['stars_x'].apply(categorize_stars)
grouped_df6 = business_categories_reviews.groupby('rating_range')['business_id'].count().reset_index()
grouped_df6 = grouped_df6.rename(columns={'business_id': 'business_count'}) 
print(grouped_df6.head())

                    rating_range  business_count
0     Average Rated (2.5 - 3.49)          180231
1         Highly Rated (4.5 - 5)          293787
2           Low Rated (1 - 2.49)           60768
3  Moderately Rated (3.5 - 4.49)          619496


In [18]:
# 9. Highest and lowest rating of each city:
business_categories_reviews_group_df = business_categories_reviews.groupby('city').agg(
    highest_rating=('stars_y', 'max'),
    lowest_rating=('stars_y', 'min')
).reset_index()
print(business_categories_reviews_group_df.head())



          city  highest_rating  lowest_rating
0     Abington             5.0            1.0
1       Affton             5.0            1.0
2        Aldan             5.0            1.0
3      Algiers             5.0            1.0
4  Aliso Viejo             5.0            5.0


In [20]:
# 10. Exploring the average yelp ratings given to different categories
santabarbara_categories_reviews_df = pd.merge(santabarbara_categories_df,dfreview, on='business_id')
santabarbara_categories_reviews_grouped_df = santabarbara_categories_reviews_df.groupby('category').agg(
avg_rating = ('stars_y', 'mean'),
business_count = ('business_id', 'nunique')).sort_values(by=['avg_rating', 'business_count'], ascending=[False, False])
print(santabarbara_categories_reviews_grouped_df)


                           avg_rating  business_count
category                                             
Coffee Roasteries            4.666667               1
Food Stands                  4.283237               3
Bakeries                     4.055002              24
Coffee & Tea                 4.025189              38
Food                         4.024329             147
Desserts                     4.002878              13
Restaurants                  3.956791             294
Ice Cream & Frozen Yogurt    3.889503              13
Bars                         3.876243              75
Food Trucks                  3.845752              12
Empanadas                    3.769231               1
Indian                       2.617021               3
Kebab                        1.428571               1
